In [1]:
from sklearn.datasets import load_boston
import torch.nn as nn
from tqdm import tqdm
import torch
import numpy as np

In [2]:
class LinearModel(nn.Module):
    # 初始化类的参数
    def __init__(self, ndim):
        super(LinearModel, self).__init__()
        # 定义一个线性层，输入维度为ndim，输出维度为1
        self.ndim = ndim
        # 初始化权重和偏置，这里因为我们是以参数的形式传入，所以我们需要使用nn.Parameter来定义
        self.weight = nn.Parameter(torch.randn(ndim, 1, device='cuda'))
        self.bias = nn.Parameter(torch.randn(1, device='cuda'))

    # 定义前向计算函数
    def forward(self, x):
        # y = Wx + b
        return x.mm(self.weight) + self.bias

In [3]:
# 加载数据
boston = load_boston()
# 初始化线性模型
lm = LinearModel(13)
# 创建损失函数
lose = nn.MSELoss()
# 初始化优化器，指定优化器的学习率为1e-16
optimizer = torch.optim.SGD(lm.parameters(),lr=1e-16)
# 提取数据并转换为tensor，并且将原始数据转换为Numpy float32类型
data = torch.tensor(boston['data'].astype(np.float32),requires_grad=True,device='cuda',dtype=torch.float32)
target = torch.tensor(boston['target'].astype(np.float32),requires_grad=True,device='cuda',dtype=torch.float32)

/home/elin/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dat

In [5]:
from tqdm import trange

In [12]:
with trange(1000) as bar:
    for i in bar:
        # 输出模型预测结果
        predict = lm(data)
        # 输出损失函数
        loseValue = lose(predict,target)
        optimizer.zero_grad() # 清零梯度
        # loss.backward()# 反向传播
        optimizer.step()
        bar.set_description("Currently Process")
        bar.set_postfix(lose=loseValue,epoch=i)
        bar.update()

  0%|          | 0/1000 [00:00<?, ?it/s]/home/elin/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([506])) that is different to the input size (torch.Size([506, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Currently Process: 100%|██████████| 1000/1000 [00:05<00:00, 174.47it/s, epoch=999, lose=tensor(3226370.5000, device='cuda:0', grad_fn=<MseLossBackward0>)]
